<a href="https://colab.research.google.com/github/csy99/dna-nn-theory/blob/master/Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl https://cog-uk.s3.climb.ac.uk/2020-05-08/cog_2020-05-08_alignment.fasta -o cog_2020-05-08_alignment.fasta
!curl https://cog-uk.s3.climb.ac.uk/2020-05-08/cog_2020-05-08_metadata.csv -o metadata.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  407M  100  407M    0     0  91.4M      0  0:00:04  0:00:04 --:--:-- 91.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  915k  100  915k    0     0  2248k      0 --:--:-- --:--:-- --:--:-- 2248k


In [ ]:
!pip install biopython

     |████████████████████████████████| 2.3MB 7.0MB/s 


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from Bio import SeqIO

In [ ]:
with open('cog_2020-05-08_alignment.fasta', 'r') as f:
  records = []
  for seq in SeqIO.parse(f, 'fasta'):
    # seq = np.char.replace(seq, '-', '')
    # seq = np.char.replace(seq, 'K', 'N')
    # seq = np.char.replace(seq, 'Y', 'N')
    # seq = np.char.replace(seq, 'R', 'N')
    # seq = np.char.replace(seq, 'W', 'N')
    # seq = np.char.replace(seq, 'M', 'N')
    # seq = np.char.replace(seq, 'S', 'N')
    records.append(" ".join([c for c in seq]))
  records = np.array(records)
  # records = np.array([record.seq._data.upper() for record in SeqIO.parse(f, 'fasta')])
seq_len = len(records[0])
print('Number of sequences :', len(records))
print('Length of sequences :', seq_len)

Number of sequences : 14277
Length of sequences : 59805


This process took 2 mins. 

In [ ]:
records[0]

'- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - A G A T C T G T T C T C T A A A C G A A C T T T A A A A T C T G T G T G G C T G T C A C T C G G C T G C A T G C T T A G T G C A C T C A C G C A G T A T A A T T A A T A A C T A A T T A C T G T C G T T G A C A G G A C A C G A G T A A C T C G T C T A T C T T C T G C A G G C T G C T T A C G G T T T C G T C C G T G T T G C A G C C G A T C A T C A G C A C A T C T A G G T T T T G T C C G G G T G T G A C C G A A A G G T A A G A T G G A G A G C C T T G T C C C T G G T T T C A A C G A G A A A A C A C A C G T C C A A C T C A G T T T G C C T G T T T T A C A G G T T C G C G A C G T G C T C G T A C G T G G C T T T G G A G A C T C C G T G G A G G A G G T C T T A T C A G A G G C A C G T C A A C A T C T T A A A G A T G G C A C T T G T G G C T T A G T A G A A G T T G A A A A A G G C G T T T T G C C T C A A C T T G A A C A G C C C T A T G T G T T C A T C A A A C G T T C G G A T G C T C G A A C T G

In [ ]:
covid_df = pd.read_csv("metadata.csv")
label = covid_df["adm1"]
label_names = covid_df["adm1"].unique()
label_names

array(['UK-NIR', 'UK-ENG', 'UK-SCT', 'UK-WLS'], dtype=object)

In [ ]:
print(covid_df["adm1"].value_counts())
print(covid_df[:2000]["adm1"].value_counts())
print(covid_df[-1000:]["adm1"].value_counts())

UK-ENG    12281
UK-WLS     2332
UK-SCT     1640
UK-NIR      127
Name: adm1, dtype: int64
UK-ENG    916
UK-WLS    709
UK-SCT    313
UK-NIR     62
Name: adm1, dtype: int64
UK-ENG    561
UK-WLS    225
UK-SCT    214
Name: adm1, dtype: int64


In [ ]:
def getWeight(label_names):
  return 1000.0/covid_df["adm1"].value_counts()[label_names]

In [ ]:
weights = []
for name in label_names:
  weight = getWeight(name)
  weights.append(weight)

weights

[7.874015748031496, 0.0814265939255761, 0.6097560975609756, 0.4288164665523156]

In [ ]:
for i in range(len(label_names)):
  label = label.replace(label_names[i], i)

In [ ]:
# should change the training/validation set size later
TRAIN_SIZE = 2000
Test_SIZE = 1000
xtrain = records[:TRAIN_SIZE]
ytrain = label[:TRAIN_SIZE]
xtest = records[-Test_SIZE:]
ytest = label[-Test_SIZE:]

In [ ]:
# ytrain[:10]

In [ ]:
seq_ds_train = tf.data.Dataset.from_tensor_slices((xtrain, ytrain))
seq_ds_train = seq_ds_train.shuffle(buffer_size=1000, seed=1).batch(8)
for x, y in seq_ds_train.take(1):
  for i in range(1):
    print(x.numpy())
    print(y.numpy())

[b'- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - A G A T C T G T T C T C T A A A C G A A C T T T A A A A T C T G T G T G G C T G T C A C T C G G C T G C A T G C T T A G T G C A C T C A C G C A G T A T A A T T A A T A A C T A A T T A C T G T C G T T G A C A G G A C A C G A G T A A C T C G T C T A T C T T C T G C A G G C T G C T T A C G G T T T C G T C C G T G T T G C A G C C G A T C A T C A G C A C A T C T A G G T T T T G T C C G G G T G T G A C C G A A A G G T A A G A T G G A G A G C C T T G T C C C T G G T T T C A A C G A G A A A A C A C A C G T C C A A C T C A G T T T G C C T G T T T T A C A G G T T C G C G A C G T G C T C G T A C G T G G C T T T G G A G A C T C C G T G G A G G A G G T C T T A T C A G A G G C A C G T C A A C A T C T T A A A G A T G G C A C T T G T G G C T T A G T A G A A G T T G A A A A A G G C G T T T T G C C T C A A C T T G A A C A G C C C T A T G T G T T C A T C A A A C G T T C G G A T G C T C G A A C T

In [ ]:
seq_ds_test = tf.data.Dataset.from_tensor_slices((xtest, ytest))
seq_ds_test = seq_ds_test.batch(8)

In [ ]:
def custom_standardization(x):
  x = tf.strings.lower(x)
  x = tf.strings.regex_replace(x, "-", "")
  x = tf.strings.regex_replace(x, 'K', 'N')
  x = tf.strings.regex_replace(x, 'Y', 'N')
  x = tf.strings.regex_replace(x, 'R', 'N')
  x = tf.strings.regex_replace(x, 'W', 'N')
  x = tf.strings.regex_replace(x, 'M', 'N')
  x = tf.strings.regex_replace(x, 'S', 'N')
  return x

In [ ]:
max_features = 20000
embedding_dim = 512
sequence_length = 1000

vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
  max_tokens=max_features,
  standardize=custom_standardization, 
  ngrams=5,
  output_mode="int",
  output_sequence_length=sequence_length
)

In [ ]:
# seq_x_ds = seq_ds_train.map(lambda x, y: x)
seq_x_partial = xtrain[:300]
vectorize_layer.adapt(seq_x_partial)
# vectorize_layer.adapt(np.array(["A T C G N"]))

In [ ]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [ ]:
seq_ds_train = seq_ds_train.map(vectorize_text)
seq_ds_train = seq_ds_train.cache().prefetch(buffer_size=10)

In [ ]:
for x, y in seq_ds_train.take(1):
  for i in range(1):
    print(x.numpy())
    print(y.numpy())

[[3 4 3 ... 2 2 2]
 [3 4 3 ... 2 2 2]
 [3 4 3 ... 2 2 2]
 ...
 [3 4 3 ... 2 2 2]
 [3 4 3 ... 2 2 2]
 [3 4 3 ... 2 2 2]]
[3 0 2 2 1 1 3 1]


In [ ]:
seq_ds_test = seq_ds_test.map(vectorize_text)
seq_ds_test = seq_ds_test.cache().prefetch(buffer_size=10)

In [ ]:
from tensorflow.keras import layers

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(4, activation="softmax", name="predictions")(x)

model1 = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model1.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer="adam", metrics=["accuracy"])

In [ ]:
class_weights = {0: weights[0], 1: weights[1], 2: weights[2], 3: weights[3]}

In [ ]:
es_cb = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
hist1 = model1.fit(seq_ds_train, epochs=15, callbacks=[es_cb])

Epoch 1/15
250/250 [==============================] - 29s 115ms/step - loss: 1.1574 - accuracy: 0.4440
Epoch 2/15
250/250 [==============================] - 30s 121ms/step - loss: 1.1175 - accuracy: 0.4595
Epoch 3/15
250/250 [==============================] - 29s 115ms/step - loss: 1.1166 - accuracy: 0.4650
Epoch 4/15
250/250 [==============================] - 30s 119ms/step - loss: 1.1201 - accuracy: 0.4595
Epoch 5/15
250/250 [==============================] - 30s 118ms/step - loss: 1.1081 - accuracy: 0.4655
Epoch 6/15
250/250 [==============================] - 28s 113ms/step - loss: 1.1094 - accuracy: 0.4650
Epoch 7/15
250/250 [==============================] - 28s 113ms/step - loss: 1.1091 - accuracy: 0.4600
Epoch 8/15
250/250 [==============================] - 28s 114ms/step - loss: 1.1113 - accuracy: 0.4645
Epoch 9/15
250/250 [==============================] - 28s 114ms/step - loss: 1.1017 - accuracy: 0.4635
Epoch 10/15
250/250 [==============================] - 29s 117ms/step - l

In [ ]:
model1.evaluate(seq_ds_train)

250/250 [==============================] - 2s 6ms/step - loss: 1.1168 - accuracy: 0.4610


[1.1167805194854736, 0.460999995470047]

In [ ]:
def evaluateModel(model, ds):
  ds_pred = np.argmax(model.predict(ds), axis=-1)
  print("prediction result shape", ds_pred.shape)
  ds_true = []
  label_ds = ds.map(lambda x, y: y)
  for item in label_ds:
    ds_true.append(item.numpy())
  ds_true = np.reshape(np.array(ds_true), (-1,)) 
  print("true label shape", ds_true.shape)
  return ds_pred, ds_true

In [ ]:
seq_ds_train_pred, seq_ds_train_true = evaluateModel(model1, seq_ds_train)

prediction result shape (2000,)
true label shape (2000,)


In [ ]:
import tensorflow_addons as tfa
confusion_mat = tfa.metrics.MultiLabelConfusionMatrix(num_classes=4)
confusion_mat.update_state(tf.one_hot(seq_ds_train_true, depth=4), tf.one_hot(seq_ds_train_pred, depth=4))

true negatives for class i in M(0,0)  
false positives for class i in M(0,1)  
false negatives for class i in M(1,0)  
true positives for class i in M(1,1)  

In [ ]:
# pd.set_option('display.float_format',lambda x : '%.2f' % x)
np.set_printoptions(suppress=True)
print('Confusion matrix:\n', confusion_mat.result().numpy())

Confusion matrix:
 [[[1922.   16.]
  [  52.   10.]]

 [[1084.    0.]
  [ 916.    0.]]

 [[1687.    0.]
  [ 313.    0.]]

 [[  26. 1265.]
  [   0.  709.]]]


In [ ]:
tmp = covid_df[:2000]["adm1"].value_counts()
# tmp["UK-NIR"]
for label in label_names:
  print(label, tmp[label])

UK-NIR 62
UK-ENG 916
UK-SCT 313
UK-WLS 709


In [ ]:
model1.evaluate(seq_ds_test)

125/125 [==============================] - 1s 6ms/step - loss: 1.0779 - accuracy: 0.5600


[1.0779014825820923, 0.5600000023841858]

In [ ]:
seq_ds_test_pred, seq_ds_test_true = evaluateModel(model1, seq_ds_test)
confusion_mat = tfa.metrics.MultiLabelConfusionMatrix(num_classes=4)
confusion_mat.update_state(tf.one_hot(seq_ds_test_pred, depth=4), tf.one_hot(seq_ds_test_true, depth=4))
print('Confusion matrix:\n', confusion_mat.result().numpy())

prediction result shape (1000,)
true label shape (1000,)
Confusion matrix:
 [[[999.   0.]
  [  1.   0.]]

 [[  0.   1.]
  [439. 560.]]

 [[786. 214.]
  [  0.   0.]]

 [[775. 225.]
  [  0.   0.]]]


Add weights for imbalanced class. 

In [ ]:
model2 = tf.keras.Sequential([                                
  layers.Embedding(max_features, embedding_dim, input_shape=[None,]),
  layers.GRU(256, return_sequences=True),  
  layers.GRU(256), 
  layers.Dense(100, activation="relu"), 
  layers.Dropout(0.3), 
  layers.Dense(4, activation="softmax")
])

In [ ]:
model2.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.SGD(momentum=0.9, nesterov=True), metrics=["accuracy"])
es_cb = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
# hist2 = model2.fit(seq_ds_train, epochs=15, callbacks=[es_cb], class_weight=class_weights)
hist2 = model2.fit(seq_ds_train, epochs=15, callbacks=[es_cb])

Epoch 1/15
250/250 [==============================] - 22s 89ms/step - loss: 1.1578 - accuracy: 0.4455
Epoch 2/15
250/250 [==============================] - 22s 89ms/step - loss: 1.1200 - accuracy: 0.4710
Epoch 3/15
250/250 [==============================] - 22s 89ms/step - loss: 1.1178 - accuracy: 0.4655
Epoch 4/15
250/250 [==============================] - 22s 89ms/step - loss: 1.1181 - accuracy: 0.4625
Epoch 5/15
250/250 [==============================] - 22s 89ms/step - loss: 1.1172 - accuracy: 0.4735
Epoch 6/15
250/250 [==============================] - 22s 89ms/step - loss: 1.1184 - accuracy: 0.4735
Epoch 7/15
250/250 [==============================] - 22s 89ms/step - loss: 1.1177 - accuracy: 0.4685
Epoch 8/15
250/250 [==============================] - 22s 89ms/step - loss: 1.1190 - accuracy: 0.4680
Epoch 9/15
250/250 [==============================] - 22s 89ms/step - loss: 1.1203 - accuracy: 0.4675
Epoch 10/15
250/250 [==============================] - 22s 89ms/step - loss: 1.120

In [ ]:
model3 = tf.keras.Sequential([                                
  layers.Embedding(max_features, embedding_dim, input_shape=[None,]),
  layers.Flatten(),
  layers.Dense(256, activation="relu"), 
  layers.Dropout(0.3),
  layers.Dense(128, activation="relu"), 
  layers.Dropout(0.3), 
  layers.Dense(32, activation="relu"), 
  layers.Dropout(0.3), 
  layers.Dense(4, activation="softmax")
])

ValueError: ignored

In [ ]:
model3.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.SGD(momentum=0.9, nesterov=True), metrics=["accuracy"])
es_cb = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
hist3 = model3.fit(seq_ds_train, epochs=15, callbacks=[es_cb], class_weight=class_weights)

Epoch 1/15


InvalidArgumentError: ignored